In [1]:
# --- Core Libraries ---
import pandas as pd
import numpy as np
import os, sys, re, joblib
from tqdm.auto import tqdm

import lightgbm as lgb 

# --- Energy imports ---
from scipy.spatial.distance import cdist
from typing import Dict, List, Tuple, Optional

# --- Helper Functions (Copied from Training Notebook) ---
USALIGN_PATH = "/home/max/Documents/Protenix-KaggleRNA3D/af3-dev/USalign/USalign"
TEMP_DIR = "./temp_pdb_inference/"
os.makedirs(TEMP_DIR, exist_ok=True)

def get_coords(df, pred_idx):
    return df[[f'x_{pred_idx}', f'y_{pred_idx}', f'z_{pred_idx}']].values

def calculate_radius_of_gyration(coords):
    center_of_mass = np.mean(coords, axis=0)
    return np.sqrt(np.mean(np.sum((coords - center_of_mass)**2, axis=1)))

def calculate_rmsd(coords1, coords2):
    from scipy.spatial.transform import Rotation as R
    if coords1.shape != coords2.shape: return np.nan
    coords1_centered = coords1 - coords1.mean(axis=0)
    coords2_centered = coords2 - coords2.mean(axis=0)
    rotation, rmsd = R.align_vectors(coords1_centered, coords2_centered)
    return rmsd

# --- Final Evaluation Script (Provided by you) ---
def parse_tmscore_output(output):
    tm_score_match = re.findall(r'TM-score=\s+([\d.]+)', output)
    if len(tm_score_match) > 1:
        return float(tm_score_match[1])
    return np.nan

def write_target_line(atom_name, atom_serial, residue_name, chain_id, residue_num,
                      x_coord, y_coord, z_coord, occupancy=1.0, b_factor=0.0, atom_type='C'):
    atom_name_padded = f" {atom_name.ljust(3)}" if len(atom_name) < 4 else atom_name
    return (
        f"ATOM  {atom_serial:5d} {atom_name_padded:<4s} {residue_name:<3s} {chain_id}"
        f"{residue_num:4d}    {x_coord:8.3f}{y_coord:8.3f}{z_coord:8.3f}"
        f"{occupancy:6.2f}{b_factor:6.2f}          {atom_type:>2s}  \n"
    )

def write2pdb(df: pd.DataFrame, xyz_id: int, target_path: str):
    resolved_cnt = 0
    written_resids = set()
    with open(target_path, 'w') as f:
        for _, row in df.iterrows():
            resid = int(row['resid'])
            if resid in written_resids: continue
            x, y, z = row.get(f'x_{xyz_id}'), row.get(f'y_{xyz_id}'), row.get(f'z_{xyz_id}')
            if pd.notna(x) and x > -1e17:
                resolved_cnt += 1
                f.write(write_target_line( "C1'", resid, row['resname'], 'A', resid, x, y, z))
                written_resids.add(resid)
    return resolved_cnt


def get_base_target_id(long_id):
    """Correctly extracts the base target ID (e.g., '9L5R_2') from a full ID ('9L5R_2_1')."""
    return "_".join(str(long_id).split("_")[:-1])

def score_and_report(solution: pd.DataFrame, submission: pd.DataFrame):
    solution['target_id'] = solution['ID'].apply(get_base_target_id)
    submission['target_id'] = submission['ID'].apply(get_base_target_id)
    
    native_idxs = sorted(int(c.split('_')[1]) for c in solution.columns if c.startswith('x_'))
    per_target, best_scores = {}, []

    for tid, grp_nat in tqdm(solution.groupby('target_id'), desc="Scoring Targets"):
        grp_pred = submission[submission['target_id'] == tid]
        if grp_pred.empty:
            print(f"Warning: No submission found for target {tid}. Skipping.")
            continue
        
        best_of_five = []
        for pred_cnt in range(1, 6):
            best_for_this_pred = 0.0
            for nat_cnt in native_idxs:
                n_nat = write2pdb(grp_nat, nat_cnt, os.path.join(TEMP_DIR, 'native.pdb'))
                n_pred = write2pdb(grp_pred, pred_cnt, os.path.join(TEMP_DIR, 'predicted.pdb'))
                if n_nat > 0 and n_pred > 0:
                    out = os.popen(f'{USALIGN_PATH} {os.path.join(TEMP_DIR, "predicted.pdb")} {os.path.join(TEMP_DIR, "native.pdb")} -atom " C1\'"').read()
                    score = parse_tmscore_output(out)
                    if score is not None:
                        best_for_this_pred = max(best_for_this_pred, score)
            best_of_five.append(best_for_this_pred)
        
        per_target[tid] = best_of_five
        best_scores.append(max(best_of_five))

    overall = np.mean(best_scores)
    print(f"\n>>> FINAL mean best-of-5 TM-score = {overall:.4f} (scored on {len(best_scores)} targets)")
    return per_target, overall

def calculate_ground_truth_tm(pred_df, pred_idx, native_df):
    """Calculates the true TM-score for one prediction against ALL possible native structures."""
    best_tm_for_this_pred = 0.0
    pred_path = os.path.join(TEMP_DIR, 'predicted_for_gt.pdb')
    native_path = os.path.join(TEMP_DIR, 'native_for_gt.pdb')
    
    n_pred = write2pdb(pred_df, pred_idx, pred_path)
    if n_pred == 0: return 0.0

    native_indices = sorted(int(c.split('_')[1]) for c in native_df.columns if c.startswith('x_'))
    for nat_idx in native_indices:
        n_nat = write2pdb(native_df, nat_idx, native_path)
        if n_nat > 0:
            cmd = f'{USALIGN_PATH} {pred_path} {native_path} -atom " C1\'"'
            output = os.popen(cmd).read()
            tm = parse_tmscore_output(output)
            if tm is not None and tm > best_tm_for_this_pred:
                best_tm_for_this_pred = tm
    return best_tm_for_this_pred

In [2]:
# --- Configuration ---
# Paths to the saved model and scaler from training
MODEL_SAVE_PATH = 'meta_learner_lgbm.pkl' 

MODEL_CONFIG = {
    'ProteinX': {
        'predictions_path': '/home/max/Documents/Standford_3DRNA_PredictionData/Protenix_Baseline_Validation/submission.csv',
        'confidences_path': '/home/max/Documents/Standford_3DRNA_PredictionData/Protenix_Baseline_Validation/confidence.csv',
        'ranking_path': '/home/max/Documents/Standford_3DRNA_PredictionData/Protenix_Baseline_Validation/ranking_scores.csv'
    },
    'DrFo2': {
        'predictions_path': '/home/max/Documents/Standford_3DRNA_PredictionData/DRfold2_Baseline_validation/submission.csv',
        'confidences_path': '/home/max/Documents/Standford_3DRNA_PredictionData/DRfold2_Baseline_validation/confidence.csv',
        'ranking_path': None
    },
    'Ribonanza': {
        # Both predictions and confidences are in this one file
        'predictions_path': '/home/max/Documents/Standford_3DRNA_PredictionData/Ribonanza_Baseline_Validation/ribonanzanet2_submission_with_confidence.csv',
        'confidences_path': '/home/max/Documents/Standford_3DRNA_PredictionData/Ribonanza_Baseline_Validation/ribonanzanet2_submission_with_confidence.csv',
        'ranking_path': None
    }
}

SEQUENCES_PATH = '/home/max/Documents/Protenix-KaggleRNA3D/data/stanford-rna-3d-folding/validation_sequences_clean.csv'
LABELS_PATH = '/home/max/Documents/Protenix-KaggleRNA3D/data/stanford-rna-3d-folding/validation_labels_clean.csv'

# Constants
NUM_PREDICTIONS_PER_MODEL = 5
NUCLEOTIDES = ['A', 'C', 'G', 'U']

print("✅ Configuration and model architecture defined.")

✅ Configuration and model architecture defined.


In [3]:
class RNAEnergyScorer:
    def __init__(self):
        # Base pairing energies (kcal/mol) at 37°C
        self.base_pair_energies = {
            ('A', 'U'): -2.0, ('U', 'A'): -2.0,  # Watson-Crick AU
            ('G', 'C'): -3.0, ('C', 'G'): -3.0,  # Watson-Crick GC
            ('G', 'U'): -1.5, ('U', 'G'): -1.5,  # Wobble GU
        }
        
        # Stacking energies for consecutive base pairs (kcal/mol)
        # Format: ((base1, base2), (base3, base4)) where 1-2 and 3-4 are pairs
        self.stacking_energies = {
            (('A', 'U'), ('A', 'U')): -0.9,
            (('A', 'U'), ('G', 'C')): -2.1,
            (('G', 'C'), ('G', 'C')): -3.4,
            (('G', 'C'), ('A', 'U')): -2.3,
            (('G', 'U'), ('G', 'U')): -1.3,
            (('G', 'U'), ('G', 'C')): -2.5,
            (('G', 'U'), ('A', 'U')): -1.4,
        }
        
        # VdW radii for RNA atoms (Angstroms)
        self.vdw_radii = {
            'C': 1.7,   # Carbon
            'N': 1.55,  # Nitrogen
            'O': 1.52,  # Oxygen
            'P': 1.8,   # Phosphorus
        }
        
        # Ideal distances for hydrogen bonds (Angstroms)
        self.hbond_ideal_distance = 2.8
        self.hbond_max_distance = 3.5
        
    def calculate_base_pair_energy(self, sequence: str, coords: np.ndarray, cutoff_distance: float = 4.0) -> float:
        if len(sequence) != len(coords):
            return 0.0
            
        energy = 0.0
        distances = cdist(coords, coords)
        
        for i in range(len(sequence)):
            for j in range(i + 3, len(sequence)):  # Skip neighbors
                if distances[i, j] < cutoff_distance:
                    pair = (sequence[i], sequence[j])
                    if pair in self.base_pair_energies:
                        # Weight by distance (closer = stronger)
                        weight = 1.0 - (distances[i, j] / cutoff_distance)
                        energy += self.base_pair_energies[pair] * weight
                        
        return energy
    
    def calculate_stacking_energy(self, sequence: str, coords: np.ndarray) -> float:
        if len(sequence) < 2 or len(coords) < 2:
            return 0.0
            
        energy = 0.0
        
        # Calculate stacking between consecutive bases
        for i in range(len(sequence) - 1):
            # Distance between consecutive bases
            dist = np.linalg.norm(coords[i] - coords[i + 1])
            
            # Ideal stacking distance is around 3.4 Angstroms
            if 3.0 < dist < 4.0:
                # Simple stacking energy based on base types
                base1, base2 = sequence[i], sequence[i + 1]
                
                # Purine-purine stacking is stronger
                if base1 in ['A', 'G'] and base2 in ['A', 'G']:
                    energy += -1.5
                # Purine-pyrimidine
                elif (base1 in ['A', 'G'] and base2 in ['C', 'U']) or \
                     (base1 in ['C', 'U'] and base2 in ['A', 'G']):
                    energy += -1.0
                # Pyrimidine-pyrimidine is weakest
                else:
                    energy += -0.5
                    
        return energy
    
    def calculate_vdw_energy(self, coords: np.ndarray, epsilon: float = 0.1, clash_penalty: float = 10.0) -> float:
        if len(coords) < 2:
            return 0.0
            
        energy = 0.0
        distances = cdist(coords, coords)
        
        # Assume all atoms are carbon for simplicity (can be extended)
        sigma = 2 * self.vdw_radii['C']
        
        for i in range(len(coords)):
            for j in range(i + 1, len(coords)):
                r = distances[i, j]
                
                if r < 0.1:  # Avoid division by zero
                    energy += clash_penalty
                elif r < sigma * 2.5:  # Only calculate for nearby atoms
                    # Lennard-Jones potential
                    ratio = sigma / r
                    energy += 4 * epsilon * (ratio**12 - ratio**6)
                    
        return energy
    
    def calculate_electrostatic_energy(self, sequence: str, coords: np.ndarray, dielectric: float = 80.0) -> float:
        if len(coords) < 2:
            return 0.0
            
        # Phosphate groups have -1 charge
        charge = -1.0
        k_e = 332.0  # Coulomb's constant in kcal*Å/(mol*e²)
        
        energy = 0.0
        distances = cdist(coords, coords)
        
        for i in range(len(coords)):
            for j in range(i + 2, len(coords)):  # Skip neighbors
                r = distances[i, j]
                if r > 0.1:  # Avoid division by zero
                    # Coulomb's law with distance-dependent dielectric
                    effective_dielectric = dielectric * (1 + 0.1 * r)
                    energy += k_e * charge * charge / (effective_dielectric * r)
                    
        return energy
    
    def calculate_hydrogen_bond_energy(self, sequence: str, coords: np.ndarray) -> float:
        if len(sequence) != len(coords): return 0.0
            
        energy = 0.0
        distances = cdist(coords, coords)
        
        # Number of H-bonds per base pair type
        hbond_counts = {
            ('A', 'U'): 2, ('U', 'A'): 2,
            ('G', 'C'): 3, ('C', 'G'): 3,
            ('G', 'U'): 2, ('U', 'G'): 2,
        }
        
        for i in range(len(sequence)):
            for j in range(i + 3, len(sequence)):
                dist = distances[i, j]
                
                if dist < self.hbond_max_distance:
                    pair = (sequence[i], sequence[j])
                    if pair in hbond_counts:
                        # Energy per H-bond with distance dependence
                        if dist <= self.hbond_ideal_distance:
                            bond_energy = -1.0  # kcal/mol per H-bond
                        else:
                            # Linear decrease from ideal to max distance
                            factor = 1 - (dist - self.hbond_ideal_distance) / \
                                    (self.hbond_max_distance - self.hbond_ideal_distance)
                            bond_energy = -1.0 * factor
                            
                        energy += bond_energy * hbond_counts[pair]
                        
        return energy
    
    def calculate_solvation_energy(self, coords: np.ndarray, 
                                  probe_radius: float = 1.4) -> float:
        if len(coords) == 0:
            return 0.0
            
        # Simplified SASA calculation
        # Count neighbors within cutoff as buried
        cutoff = 5.0  # Angstroms
        distances = cdist(coords, coords)
        
        energy = 0.0
        for i in range(len(coords)):
            # Count neighbors
            neighbors = np.sum((distances[i] > 0) & (distances[i] < cutoff))
            
            # More neighbors = more buried = less solvation penalty
            # Assume -0.5 kcal/mol per exposed nucleotide
            exposure = max(0, 1 - neighbors / 6.0)  # 6 is roughly fully buried
            energy += -0.5 * exposure
            
        return energy
    
    def calculate_torsion_energy(self, coords: np.ndarray) -> float:
        if len(coords) < 4:
            return 0.0
            
        energy = 0.0
        
        # Calculate pseudo-torsion angles for consecutive 4 nucleotides
        for i in range(len(coords) - 3):
            # Get 4 consecutive points
            p1, p2, p3, p4 = coords[i:i+4]
            
            # Calculate vectors
            v1 = p2 - p1
            v2 = p3 - p2
            v3 = p4 - p3
            
            # Calculate normal vectors to planes
            n1 = np.cross(v1, v2)
            n2 = np.cross(v2, v3)
            
            # Avoid division by zero
            if np.linalg.norm(n1) > 0 and np.linalg.norm(n2) > 0:
                n1 = n1 / np.linalg.norm(n1)
                n2 = n2 / np.linalg.norm(n2)
                
                # Calculate dihedral angle
                cos_angle = np.clip(np.dot(n1, n2), -1, 1)
                angle = np.arccos(cos_angle)
                
                # Simple torsion potential (favors certain angles)
                # Minima at 0, π/3, 2π/3, π, 4π/3, 5π/3
                energy += 0.5 * (1 + np.cos(3 * angle))
                
        return energy
    
    def calculate_total_energy(self, sequence: str, coords: np.ndarray,
                             weights: Optional[Dict[str, float]] = None) -> Dict[str, float]:
        if weights is None:
            weights = {
                'base_pair': 1.0,
                'stacking': 0.5,
                'vdw': 0.3,
                'electrostatic': 0.2,
                'hbond': 0.8,
                'solvation': 0.4,
                'torsion': 0.3
            }
        
        energies = {
            'base_pair': self.calculate_base_pair_energy(sequence, coords),
            'stacking': self.calculate_stacking_energy(sequence, coords),
            'vdw': self.calculate_vdw_energy(coords),
            'electrostatic': self.calculate_electrostatic_energy(sequence, coords),
            'hbond': self.calculate_hydrogen_bond_energy(sequence, coords),
            'solvation': self.calculate_solvation_energy(coords),
            'torsion': self.calculate_torsion_energy(coords)
        }
        
        # Calculate weighted total
        total = sum(weights.get(key, 1.0) * value for key, value in energies.items())
        energies['total'] = total
        
        return energies


def extract_energy_features(sequence: str, coords: np.ndarray, scorer: Optional[RNAEnergyScorer] = None) -> Dict[str, float]:
    if scorer is None: scorer = RNAEnergyScorer()
    
    energies = scorer.calculate_total_energy(sequence, coords)
    features = energies.copy()
    
    # Energy per nucleotide
    n_nucleotides = len(sequence)
    if n_nucleotides > 0:
        features['energy_per_nucleotide'] = energies['total'] / n_nucleotides
        features['base_pair_per_nucleotide'] = energies['base_pair'] / n_nucleotides
    
    # Ratio features
    if energies['total'] != 0:
        features['base_pair_ratio'] = energies['base_pair'] / abs(energies['total'])
        features['hbond_ratio'] = energies['hbond'] / abs(energies['total'])
    
    # Stability score (lower energy = more stable)
    features['stability_score'] = -energies['total']
    
    return features

In [4]:
# --- Step 1: Load and Pre-process All Data ---
print("--- Step 1: Loading and Pre-processing All Data ---")
try:
    df_sequences = pd.read_csv(SEQUENCES_PATH)
    df_labels = pd.read_csv(LABELS_PATH)
    df_labels['target_id'] = df_labels['ID'].apply(get_base_target_id)
    
    data_dfs = {}
    for model_name, config in MODEL_CONFIG.items():
        if model_name == 'Ribonanza':
            df_raw = pd.read_csv(config['predictions_path'])
            pred_cols = ['ID', 'resname', 'resid'] + [col for col in df_raw.columns if col.startswith(('x_', 'y_', 'z_'))]
            data_dfs[f'{model_name}_preds'] = df_raw[pred_cols].copy()
            rename_dict = {f'confidence_{i}': f'plddt_{i}' for i in range(1, NUM_PREDICTIONS_PER_MODEL + 1)}
            data_dfs[f'{model_name}_conf'] = df_raw.rename(columns=rename_dict)
        else:
            data_dfs[f'{model_name}_preds'] = pd.read_csv(config['predictions_path'])
            data_dfs[f'{model_name}_conf'] = pd.read_csv(config['confidences_path'])
        
        if config.get('ranking_path'):
            data_dfs[f'{model_name}_rank'] = pd.read_csv(config['ranking_path'])

    print("All data files loaded.")
except FileNotFoundError as e:
    print(f"FATAL ERROR: Cannot load a data file: {e}.")
    raise e

# --- Step 2: Main Feature Generation Loop with Integrated Checks ---
print("\n--- Step 2: Generating Features for Inference ---")
energy_scorer = RNAEnergyScorer()
meta_data_rows = []
all_target_ids_in_sequence_file = df_sequences['target_id'].unique()
print(f"Found {len(all_target_ids_in_sequence_file)} unique targets in the primary sequence file.")

for target_id in tqdm(all_target_ids_in_sequence_file, desc="Processing Targets"):
    
    # --- Robustness Check: Ensure this target exists where needed ---
    sequence_row = df_sequences[df_sequences['target_id'] == target_id]
    if sequence_row.empty: continue # Should not happen, but safe
    
    # Check if this target_id has predictions from ALL models
    is_present_in_all = True
    for model_name in MODEL_CONFIG.keys():
        if data_dfs[f'{model_name}_preds'][data_dfs[f'{model_name}_preds']['ID'].str.startswith(target_id)].empty:
            is_present_in_all = False
            # print(f"Warning: Skipping {target_id}, missing predictions from {model_name}")
            break
    if not is_present_in_all: continue

    # --- If checks pass, proceed with feature generation ---
    sequence = sequence_row.iloc[0]['sequence']
    
    # Pre-calculate all candidate coordinates for this target
    all_candidate_coords = {}
    for model_name in MODEL_CONFIG:
        target_preds = data_dfs[f'{model_name}_preds'][data_dfs[f'{model_name}_preds']['ID'].str.startswith(target_id)]
        for i in range(1, NUM_PREDICTIONS_PER_MODEL + 1):
            all_candidate_coords[f'{model_name}_{i}'] = get_coords(target_preds, i)

    # Loop through each model to create feature rows
    for model_name in MODEL_CONFIG:
        df_conf = data_dfs[f'{model_name}_conf']
        target_confs = df_conf[df_conf['ID'].str.startswith(target_id)]
        
        df_rank = data_dfs.get(f'{model_name}_rank')
        target_rank = df_rank[df_rank['target_id'] == target_id] if df_rank is not None else None

        for i in range(1, NUM_PREDICTIONS_PER_MODEL + 1):
            if f'plddt_{i}' not in target_confs.columns: continue
            
            features = {
                'target_id': target_id, 
                'model_source_str': model_name,
                'prediction_index': i
            }
            
            features['sequence_length'] = len(sequence)
            for nuc in NUCLEOTIDES: features[f'percent_{nuc}'] = sequence.count(nuc) / len(sequence)
            
            plddt_scores = target_confs[f'plddt_{i}'].values
            features.update({'mean_plddt': np.mean(plddt_scores), 'std_plddt': np.std(plddt_scores), 'min_plddt': np.min(plddt_scores), 'percent_low_conf': np.mean(plddt_scores < 70), 'percent_high_conf': np.mean(plddt_scores > 90)})
            
            if target_rank is not None and not target_rank.empty:
                features.update({'ptm': target_rank.iloc[0].get(f'ptm_{i}', 0), 'ranking_score': target_rank.iloc[0].get(f'ranking_score_{i}', 0)})
            else:
                features.update({'ptm': 0, 'ranking_score': 0})

            candidate_coords = all_candidate_coords.get(f'{model_name}_{i}')
            if candidate_coords is None or candidate_coords.shape[0] == 0: continue
            features['radius_of_gyration'] = calculate_radius_of_gyration(candidate_coords)
            rmsd_to_others = [calculate_rmsd(candidate_coords, other_coords) for key, other_coords in all_candidate_coords.items() if key != f'{model_name}_{i}']
            features['avg_rmsd_to_others'] = np.nanmean(rmsd_to_others)
            
            base_mean_plddt = features['mean_plddt']
            base_std_plddt = features['std_plddt']
            base_radius_of_gyration = features['radius_of_gyration']
            for source in MODEL_CONFIG.keys():
                features[f'plddt_x_{source}'] = 0.0
                features[f'std_plddt_x_{source}'] = 0.0
                features[f'rog_x_{source}'] = 0.0
            features[f'plddt_x_{model_name}'] = base_mean_plddt
            features[f'std_plddt_x_{model_name}'] = base_std_plddt
            features[f'rog_x_{model_name}'] = base_radius_of_gyration
            
            try:
                energy_features = extract_energy_features(sequence, candidate_coords, energy_scorer)
                features.update({
                    'energy_total': energy_features['total'],
                    'energy_base_pair': energy_features['base_pair'],
                    'energy_stacking': energy_features['stacking'],
                    'energy_vdw': energy_features['vdw'],
                    'energy_electrostatic': energy_features['electrostatic'],
                    'energy_hbond': energy_features['hbond'],
                    'energy_solvation': energy_features['solvation'],
                    'energy_torsion': energy_features['torsion'],
                    'energy_per_nucleotide': energy_features.get('energy_per_nucleotide', 0),
                    'energy_stability_score': energy_features.get('stability_score', 0),
                    'energy_base_pair_ratio': energy_features.get('base_pair_ratio', 0),
                    'energy_hbond_ratio': energy_features.get('hbond_ratio', 0)
                })
                attractive_energy = energy_features['base_pair'] + energy_features['stacking'] + energy_features['hbond']
                repulsive_energy = energy_features['vdw'] + energy_features['electrostatic']
                features['energy_attractive'] = attractive_energy
                features['energy_repulsive'] = repulsive_energy
                features['energy_balance'] = attractive_energy / (abs(repulsive_energy) + 1e-6)
                features['energy_density'] = energy_features['total'] / (len(sequence) + 1e-6)
                energy_components = [energy_features[k] for k in ['base_pair', 'stacking', 'vdw', 'electrostatic', 'hbond', 'solvation', 'torsion']]
                features['energy_variance'] = np.var(energy_components)
                features['energy_std'] = np.std(energy_components)
            except Exception as e:
                # print(f"Warning: Error calculating energy for {target_id}_{model_name}_{i}: {e}")
                energy_feature_names = [
                    'energy_total', 'energy_base_pair', 'energy_stacking', 'energy_vdw',
                    'energy_electrostatic', 'energy_hbond', 'energy_solvation', 'energy_torsion',
                    'energy_per_nucleotide', 'energy_stability_score', 'energy_base_pair_ratio',
                    'energy_hbond_ratio', 'energy_attractive', 'energy_repulsive', 'energy_balance',
                    'energy_density', 'energy_variance', 'energy_std'
                ]
                for feat_name in energy_feature_names:
                    features[feat_name] = 0.0
                    
            meta_data_rows.append(features)

# --- Step 3: Finalize DataFrame ---
if not meta_data_rows:
    raise ValueError("FATAL ERROR: No feature rows were generated. Check for data mismatches or empty files.")

df_inference = pd.DataFrame(meta_data_rows)

# Create the numeric 'model_source' column that the model was trained on
model_mapping = {name: i for i, name in enumerate(MODEL_CONFIG.keys())}
df_inference['model_source'] = df_inference['model_source_str'].map(model_mapping)

df_inference.fillna(df_inference.median(numeric_only=True), inplace=True)

print(f"\n✅ Feature generation complete for {df_inference['target_id'].nunique()} targets.")
display(df_inference.head())

print(f"Unique targets in SEQUENCES file after filtering: {df_sequences['target_id'].nunique()}")
print(f"Unique targets in LABELS file after filtering: {df_labels['target_id'].nunique()}")

--- Step 1: Loading and Pre-processing All Data ---
All data files loaded.

--- Step 2: Generating Features for Inference ---
Found 94 unique targets in the primary sequence file.


Processing Targets:   0%|          | 0/94 [00:00<?, ?it/s]


✅ Feature generation complete for 94 targets.


,target_id,model_source_str,prediction_index,sequence_length,percent_A,percent_C,percent_G,percent_U,mean_plddt,std_plddt,...,energy_stability_score,energy_base_pair_ratio,energy_hbond_ratio,energy_attractive,energy_repulsive,energy_balance,energy_density,energy_variance,energy_std,model_source
0,9L5R_2,ProteinX,1,193,0.170984,0.26943,0.217617,0.341969,50.865285,6.674131,...,-111.026941,0.000000,0.000000,0.000000,558.202298,0.000000,0.575269,41774.617491,204.388399,0
1,9L5R_2,ProteinX,2,193,0.170984,0.26943,0.217617,0.341969,50.863990,6.291851,...,-104.898808,0.000000,0.000000,0.000000,534.964876,0.000000,0.543517,38639.473721,196.569259,0
2,9L5R_2,ProteinX,3,193,0.170984,0.26943,0.217617,0.341969,50.045337,6.346398,...,-111.859021,0.000000,0.000000,0.000000,560.106704,0.000000,0.579580,41984.046444,204.900089,0
3,9L5R_2,ProteinX,4,193,0.170984,0.26943,0.217617,0.341969,51.018135,6.726162,...,-109.998523,0.000000,0.000000,0.000000,559.019174,0.000000,0.569941,41867.415541,204.615287,0
4,9L5R_2,ProteinX,5,193,0.170984,0.26943,0.217617,0.341969,50.661917,6.356630,...,-117.409530,-0.006255,-0.017034,-2.734425,593.812545,-0.004605,0.608340,44944.874678,212.002063,0


Unique targets in SEQUENCES file after filtering: 94
Unique targets in LABELS file after filtering: 94


In [5]:
print("\n--- Loading LightGBM model and making predictions... ---")

# Load the trained LightGBM model
try:
    model = joblib.load(MODEL_SAVE_PATH)
except FileNotFoundError:
    print(f"FATAL ERROR: Model file not found at {MODEL_SAVE_PATH}. Please run the training notebook first.")
    raise

# Define the feature columns (must match the training script)
feature_cols = [col for col in df_inference.columns if col not in ['target_id', 'model_source_str', 'tm_score']] # tm_score might not exist
X_inference = df_inference[feature_cols].copy()

# Clean column names to be compatible with LightGBM (must be identical to training)
X_inference.columns = ["".join (c if c.isalnum() else "_" for c in str(x)) for x in X_inference.columns]

# --- Sanity Check: Ensure feature order is the same as in the trained model ---
try:
    X_inference = X_inference[model.feature_name_]
except Exception as e:
    print(f"FATAL ERROR: Feature mismatch between inference data and trained model. Error: {e}")
    print("\nFeatures in inference data:", X_inference.columns.tolist())
    print("Features expected by model:", model.feature_name_)
    raise

# Make predictions
# No scaling or tensor conversion needed!
predicted_scores = model.predict(X_inference)

# Add predictions to the dataframe
df_inference['predicted_tm'] = predicted_scores

# --- Core Selection Logic ---
# For each target_id, find the 5 candidates with the highest predicted score
df_top5 = df_inference.groupby('target_id').apply(lambda x: x.nlargest(5, 'predicted_tm')).reset_index(drop=True)

print("✅ Top 5 candidates selected for each target.")
print(f"Total selected structures: {len(df_top5)}")
display(df_top5[['target_id', 'model_source_str', 'prediction_index', 'predicted_tm']].head(10))


--- Loading LightGBM model and making predictions... ---
✅ Top 5 candidates selected for each target.
Total selected structures: 470


/tmp/ipykernel_183721/3663936403.py:35: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_top5 = df_inference.groupby('target_id').apply(lambda x: x.nlargest(5, 'predicted_tm')).reset_index(drop=True)


,target_id,model_source_str,prediction_index,predicted_tm
0,8K85_A,Ribonanza,3,0.585564
1,8K85_A,Ribonanza,4,0.577453
2,8K85_A,Ribonanza,5,0.575787
3,8K85_A,Ribonanza,2,0.570707
4,8K85_A,Ribonanza,1,0.569004
5,8KEB_A,Ribonanza,4,0.882930
6,8KEB_A,Ribonanza,5,0.878771
7,8KEB_A,ProteinX,5,0.669664
8,8KEB_A,ProteinX,2,0.659660
9,8KEB_A,ProteinX,4,0.658660


In [6]:
# --- Step 3: Assemble the Ensembled Submission File (CORRECTED) ---
print("\n--- Assembling final submission file... ---")

final_submission_rows = []
# Group by target_id to process one RNA at a time
for target_id, group in tqdm(df_top5.groupby('target_id'), desc="Assembling Submission"):
    
    # Get the base information (ID, resname, resid) from any of the original prediction files.
    base_info_df = None
    for model_name in MODEL_CONFIG.keys():
        base_info = data_dfs[f'{model_name}_preds'][data_dfs[f'{model_name}_preds']['ID'].str.startswith(f"{target_id}_")]
        if not base_info.empty:
            base_info_df = base_info[['ID', 'resname', 'resid']].copy()
            break
            
    if base_info_df is None:
        print(f"Warning: Could not find base info for target {target_id}. Skipping.")
        continue
    
    target_submission_df = base_info_df.set_index('resid')
    
    # Iterate from k=1 to 5 to create the new columns x_1, y_1, z_1 ... x_5, y_5, z_5
    for k, (_, selection_row) in enumerate(group.iterrows(), 1):
        
        source_model = selection_row['model_source_str']
        original_pred_idx = selection_row['prediction_index']
        
        # Get the original coordinate data for this selection
        original_coords_df = data_dfs[f'{source_model}_preds'][data_dfs[f'{source_model}_preds']['ID'].str.startswith(f"{target_id}_")]
        
        # Prepare the coordinates to add, with 'resid' as the index
        coords_to_add = original_coords_df[['resid', f'x_{original_pred_idx}', f'y_{original_pred_idx}', f'z_{original_pred_idx}']].copy()
        coords_to_add = coords_to_add.set_index('resid')
        
        # Assign the new columns. This is safer than merging.
        target_submission_df[f'x_{k}'] = coords_to_add[f'x_{original_pred_idx}']
        target_submission_df[f'y_{k}'] = coords_to_add[f'y_{original_pred_idx}']
        target_submission_df[f'z_{k}'] = coords_to_add[f'z_{original_pred_idx}']
        
    # Reset the index to make 'resid' a column again, matching the submission format
    final_submission_rows.append(target_submission_df.reset_index())

# Concatenate all targets into the final submission dataframe
if final_submission_rows:
    ensembled_submission_df = pd.concat(final_submission_rows)
    ensembled_submission_df.to_csv('ensembled_submission.csv', index=False)
    print("Ensembled submission file 'ensembled_submission.csv' created successfully.")
    display(ensembled_submission_df.head())
else:
    print("ERROR: No data was assembled for the final submission.")


--- Assembling final submission file... ---


Assembling Submission:   0%|          | 0/94 [00:00<?, ?it/s]

Ensembled submission file 'ensembled_submission.csv' created successfully.


,resid,ID,resname,x_1,y_1,z_1,x_2,y_2,z_2,x_3,y_3,z_3,x_4,y_4,z_4,x_5,y_5,z_5
0,1,8K85_A_1,G,11.359874,19.273932,-24.725073,-10.031795,21.430502,24.104576,1.739657,25.807976,-21.325783,-25.030249,-3.009271,22.493553,-11.122301,25.338013,20.354237
1,2,8K85_A_2,G,11.292404,13.435502,-24.485357,-12.683558,19.124477,19.230865,2.243749,20.615433,-21.924036,-22.164127,-6.809379,19.551247,-6.083194,22.807205,19.369934
2,3,8K85_A_3,A,12.951257,9.367302,-22.206780,-13.850716,18.713408,14.441497,4.780207,16.301336,-21.403898,-21.007872,-9.346490,15.168128,-2.446923,19.225378,19.853275
3,4,8K85_A_4,G,15.312449,6.391326,-18.453760,-12.531183,18.946608,9.559502,8.139541,12.631317,-19.601051,-20.436550,-9.683000,9.947579,-0.673806,14.225980,20.273897
4,5,8K85_A_5,A,17.220327,4.177942,-13.849748,-10.470436,19.030570,4.495241,11.442865,9.733561,-16.639530,-20.069801,-9.160627,4.501581,0.640619,8.873339,20.482450


In [7]:
# --- Step 4: Evaluate the Final Result ---
print("--- Evaluating the performance of the meta-learner ensemble... ---")

# Load the ground truth labels
solution_df = pd.read_csv(LABELS_PATH)

# The submission dataframe is the one we just created
submission_df = pd.read_csv('ensembled_submission.csv')

# Run the scoring
_, final_score = score_and_report(solution_df, submission_df)

--- Evaluating the performance of the meta-learner ensemble... ---


Scoring Targets:   0%|          | 0/94 [00:00<?, ?it/s]


>>> FINAL mean best-of-5 TM-score = 0.4750 (scored on 94 targets)


In [ ]:
print("\n--- In-Depth Ensemble Performance Analysis ---")

# Step 1: Calculate the true TM-score for every candidate in df_inference.
print("Calculating ground truth TM-scores for analysis (efficiently)...")

# This list will store small dataframes, each with results for one target
all_target_results = []

# Get the list of unique targets from the inference results
unique_targets_to_analyze = df_inference['target_id'].unique()

for target_id in tqdm(unique_targets_to_analyze, desc="Calculating True TM-scores"):
    # Get the slice of predictions for this one target
    inference_slice = df_inference[df_inference['target_id'] == target_id].copy()
    
    # Get the corresponding labels once for this target
    native_df_for_target = df_labels[df_labels['ID'].apply(get_base_target_id) == target_id]
    
    # This list will hold the calculated true TM-scores for this target
    true_scores = []
    
    # Iterate through the rows of the slice (15 rows per target)
    for _, row in inference_slice.iterrows():
        model_name = row['model_source_str']
        pred_idx = row['prediction_index']
        
        # Get the prediction data for the specific model
        pred_df_for_target = data_dfs[f'{model_name}_preds'][data_dfs[f'{model_name}_preds']['ID'].apply(get_base_target_id) == target_id]

        if native_df_for_target.empty or pred_df_for_target.empty:
            tm_score = np.nan
        else:
            tm_score = calculate_ground_truth_tm(pred_df_for_target, pred_idx, native_df_for_target)
            
        true_scores.append(tm_score)
        
    # Add the list of scores as a new column to the slice
    inference_slice['true_tm_score'] = true_scores
    all_target_results.append(inference_slice)

# Concatenate all the small dataframes back into one
df_analysis = pd.concat(all_target_results)

print("✅ Analysis data prepared successfully.")

# --- DEBUGGING CHECK ---
if df_analysis['true_tm_score'].isnull().all():
    print("\n❌ CRITICAL ERROR: Could not calculate any true TM-scores.")
else:
    print("\n✅ True TM-scores calculated and merged successfully.")
    
    # --- Metric 1: "Oracle Score" ---
    oracle_best_scores = df_analysis.groupby('target_id')['true_tm_score'].max()
    oracle_score = oracle_best_scores.mean()
    print(f"1. Oracle Score (Theoretical Max): {oracle_score:.4f}")

    # --- Metric 2: Your Model's Achieved Score ---
    # Assumes 'final_score' was correctly calculated in the cell above.
    print(f"2. Your Model's Achieved Score:     {final_score:.4f}")
    if oracle_score > 0:
        print(f"   (Your model achieved {final_score/oracle_score:.2%} of the theoretical maximum performance)")

    # --- Metric 3: Recall@5 ---
    recall_hits = 0
    total_targets = df_analysis['target_id'].nunique()
    for target_id, group in df_analysis.groupby('target_id'):
        if group['true_tm_score'].notna().any():
            best_candidate_true_idx = group['true_tm_score'].idxmax()
            top_5_predicted_indices = group.nlargest(5, 'predicted_tm').index
            if best_candidate_true_idx in top_5_predicted_indices:
                recall_hits += 1

    if total_targets > 0:
        recall_at_5 = recall_hits / total_targets
        print(f"\n3. Recall@5 (Found the single best candidate): {recall_at_5:.2%} ({recall_hits}/{total_targets} targets)")

    # --- Metric 4: Distribution of selected models ---
    print("\n4. Distribution of Models in Your Final Top 5 Selection:")
    model_distribution = df_top5['model_source_str'].value_counts(normalize=True)
    print(model_distribution)

    # --- Metric 5: Average Predicted vs. True TM-score for each model ---
    print("\n5. Model Performance Analysis (Avg Predicted vs. True TM-score):")
    analysis_summary = df_analysis.groupby('model_source_str').agg(
        avg_predicted_tm=('predicted_tm', 'mean'),
        avg_true_tm=('true_tm_score', 'mean'),
        count=('target_id', 'size')
    ).sort_values(by='avg_true_tm', ascending=False)
    print(analysis_summary)


--- In-Depth Ensemble Performance Analysis ---
Calculating ground truth TM-scores for analysis (efficiently)...


Calculating True TM-scores:   0%|          | 0/94 [00:00<?, ?it/s]

✅ Analysis data prepared successfully.

✅ True TM-scores calculated and merged successfully.
1. Oracle Score (Theoretical Max): 0.5021
2. Your Model's Achieved Score:     0.4750
   (Your model achieved 94.62% of the theoretical maximum performance)

3. Recall@5 (Found the single best candidate): 53.19% (50/94 targets)

4. Distribution of Models in Your Final Top 5 Selection:
model_source_str
Ribonanza    0.534043
ProteinX     0.300000
DrFo2        0.165957
Name: proportion, dtype: float64

5. Model Performance Analysis (Avg Predicted vs. True TM-score):
                  avg_predicted_tm  avg_true_tm  count
model_source_str                                      
ProteinX                  0.406427     0.403984    470
DrFo2                     0.360612     0.353896    470
Ribonanza                 0.433002     0.319214    470
